In [1]:
import pandas as pd
import numpy as np

In [23]:
all_transactions = pd.read_csv('all_transactions.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
all_transactions['value'] = all_transactions['value'].astype('float64')

In [4]:
all_transactions.columns

Index(['from', 'to', 'value', 'time', 'block'], dtype='object')

In [5]:
trans_genesis = all_transactions[all_transactions['block']==0]

In [7]:
trans_genesis.head()

,from,to,value,time,block
0,ethereum,0000000000000000000000000000000000000000,5.000000e+18,0,0
204952137,genesis,756f45e3fa69347a9a973a725e3c98bc4db0b5a0,2.000000e+20,0,0
204952138,genesis,f42f905231c770f0a406f2b768877fb49eee0f21,1.970000e+20,0,0
204952139,genesis,2489ac126934d4d6a94df08743da7b7691e9798e,1.000000e+21,0,0
204952140,genesis,ddf5810a0eb2fb2e32323bb2c99509ab320f24ac,1.790000e+22,0,0


In [8]:
val = trans_genesis['value'].sum()

In [9]:
from web3 import Web3

In [10]:
Web3.fromWei(val, 'ether')

Decimal('72009995.499480001716158464')

In [29]:
# a dictionary mapping address to the corresponding genesis level
from multiprocessing import Process,Manager
manager = Manager()
genesisLevel = manager.dict()

In [18]:
curr_level = 0
filtered_trans = all_transactions[(all_transactions['from'] == 'ethereum') | (all_transactions['from'] == 'genesis')]

In [20]:
filtered_trans['from'].unique()

array(['ethereum', 'genesis'], dtype=object)

In [21]:
# gather all addresses
addresses = filtered_trans['to'].unique()

In [23]:
len(addresses)

13044

In [24]:
reward_transaction = all_transactions[all_transactions['from']=='ethereum']

In [25]:
reward_transaction.count()

from     6458881
to       6458881
value    6458881
time     6458881
block    6458881
dtype: int64

In [28]:
len(reward_transaction['to'].unique())

4176

In [32]:
x = [1,2,3,4,5,6,7,8,9]
print(x[0:3])
print(x[3:6])
print(x[6:])


[1, 2, 3]
[4, 5, 6]
[7, 8, 9]


In [33]:
curr_level = 1

In [37]:
partition_size = int(len(addresses)/3) + 1
start_i = 0
end_i = start_i + partition_size
for i in range(3):
    if i==2:
        print(len(addresses[start_i:]))
    else:
        print(len(addresses[start_i:end_i]))
    start_i = end_i
    end_i = start_i + partition_size
print(len(addresses))    

4349
4349
4346
13044


In [24]:
# fill dictionary
def fillLevels(address_list, genesisLevel, curr_level):
    for addr in address_list:
        genesisLevel[addr] = curr_level

In [40]:
len(genesisLevel)

13044

In [47]:
filtered_trans = all_transactions[all_transactions['from'].isin(addresses) ]['to']

In [49]:
len(filtered_trans.unique())

990626

In [52]:
len(addresses)

13044

In [25]:
from multiprocessing import Process,Manager
manager = Manager()
genesisLevel = manager.dict()
curr_level = 0

In [26]:
filtered_trans = all_transactions[(all_transactions['from'] == 'ethereum') | (all_transactions['from'] == 'genesis')]['to']
addresses = filtered_trans.unique()

In [13]:
new_transactions = all_transactions[~all_transactions['to'].isin(addresses)]

In [21]:
all_transactions.count()

from     204961030
to       204961030
value    204961030
time     204961030
block    204961030
dtype: int64

In [15]:
new_transactions.count()

from     198182969
to       198182969
value    198182969
time     198182969
block    198182969
dtype: int64

In [27]:
while len(addresses)>0:
    curr_level+=1
    print('looking into level: ',curr_level)
    jobs = []
    job_cnt = 4
    partition_size = int(len(addresses)/job_cnt) + 1
    start_i = 0
    end_i = start_i + partition_size
    for i in range(job_cnt):
        if i==job_cnt-1:
            p = Process(target=fillLevels, args=(addresses[start_i:],genesisLevel,curr_level))
        else:
            p = Process(target=fillLevels, args=(addresses[start_i:end_i],genesisLevel,curr_level))
        start_i = end_i
        end_i = start_i + partition_size
        jobs.append(p)
        p.start()

    for proc in jobs:
        proc.join()
    
    print('filled the dictionary level values')
    all_transactions = all_transactions[~all_transactions['to'].isin(addresses)]
    filtered_trans = all_transactions[all_transactions['from'].isin(addresses)]['to']
    addresses = filtered_trans.unique()
    #addresses = np.setdiff1d(new_addresses,addresses) # remove addresses of higher level
    print('obtained addresses for the next level')
    print('addresses obtained: ',len(addresses))

looking into level:  1
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  988426
looking into level:  2
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  4708611
looking into level:  3
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  8533293
looking into level:  4
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  3331247
looking into level:  5
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1697040
looking into level:  6
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1635002
looking into level:  7
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1917090
looking into level:  8
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  1949139
l

filled the dictionary level values
obtained addresses for the next level
addresses obtained:  228
looking into level:  69
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  226
looking into level:  70
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  220
looking into level:  71
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  217
looking into level:  72
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  209
looking into level:  73
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  203
looking into level:  74
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  198
looking into level:  75
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  200
looking into level:  76
filled the dictionary le

obtained addresses for the next level
addresses obtained:  64
looking into level:  136
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  62
looking into level:  137
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  60
looking into level:  138
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  59
looking into level:  139
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  59
looking into level:  140
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  59
looking into level:  141
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  58
looking into level:  142
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  58
looking into level:  143
filled the dictionary level values
obtained addresses for t

filled the dictionary level values
obtained addresses for the next level
addresses obtained:  25
looking into level:  204
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  25
looking into level:  205
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  25
looking into level:  206
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  25
looking into level:  207
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  24
looking into level:  208
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  24
looking into level:  209
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  24
looking into level:  210
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  24
looking into level:  211
filled the dictionary le

obtained addresses for the next level
addresses obtained:  15
looking into level:  271
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  15
looking into level:  272
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  15
looking into level:  273
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  15
looking into level:  274
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  15
looking into level:  275
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  15
looking into level:  276
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  15
looking into level:  277
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  15
looking into level:  278
filled the dictionary level values
obtained addresses for t

...

obtained addresses for the next level
addresses obtained:  2
looking into level:  4595
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4596
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4597
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4598
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4599
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4600
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4601
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4602
filled the dictionary level values
obtained addresses for t

filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4663
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4664
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4665
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4666
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4667
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4668
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4669
filled the dictionary level values
obtained addresses for the next level
addresses obtained:  2
looking into level:  4670
filled the dictionary le

Process Process-18800:
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/multiprocessing/managers.py", line 749, in _callmethod
    conn = self._tls.connection
AttributeError: 'ForkAwareLocal' object has no attribute 'connection'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()


KeyboardInterrupt: 

  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-24-9ac3d767f44f>", line 4, in fillLevels
    genesisLevel[addr] = curr_level
  File "<string>", line 2, in __setitem__
  File "/anaconda3/lib/python3.6/multiprocessing/managers.py", line 753, in _callmethod
    self._connect()
  File "/anaconda3/lib/python3.6/multiprocessing/managers.py", line 740, in _connect
    conn = self._Client(self._token.address, authkey=self._authkey)
  File "/anaconda3/lib/python3.6/multiprocessing/connection.py", line 487, in Client
    c = SocketClient(address)
  File "/anaconda3/lib/python3.6/multiprocessing/connection.py", line 614, in SocketClient
    s.connect(address)
FileNotFoundError: [Errno 2] No such file or directory


In [29]:
all_transactions.count()

from     273751
to       273751
value    273751
time     273751
block    273751
dtype: int64

In [31]:
genesisLevel

<DictProxy object, typeid 'dict' at 0xf906bbb70>

In [34]:
for k in genesisLevel.keys():
    print(k,genesisLevel[k])
    break

FileNotFoundError: [Errno 2] No such file or directory